In [1]:
!pip install groq python-dotenv numpy tqdm datasets

In [2]:
from groq import Groq
from dotenv import load_dotenv
from datasets import load_dataset

import os
from tqdm import tqdm
import re
import random
import pprint

from typing import List, Dict, Any

load_dotenv()
random.seed(0)

client = Groq()
gsm8k_dataset = load_dataset("gsm8k", "main")

gsm8k_train = gsm8k_dataset["train"]
gsm8k_test  = gsm8k_dataset["test"]

In [3]:
def generate_response_using_Llama(
        prompt: str,
        model: str = "llama3-8b-8192"
    ):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant that solves math problems."
                },
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            model=model,
            temperature=0.3, ### 수정해도 됩니다!
            stream=False
        )
        return chat_completion.choices[0].message.content
    
    except Exception as e:
        print(f"API call error: {str(e)}")
        return None

#### 응답 잘 나오는지 확인해보기

In [4]:
response = generate_response_using_Llama(
    prompt="Hello world!",
)
print(response)

Hello! I'm happy to help you with any math problems you have. What kind of math are you working on? Do you have a specific problem you're stuck on or a concept you're trying to understand? Let me know and I'll do my best to assist you!


#### GSM8K 데이터셋 확인해보기

In [5]:
print("[Question]")
for l in gsm8k_test['question'][0].split("."):
    print(l)
print("="*100)
print("[Answer]")
print(gsm8k_test['answer'][0])

[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


#### Util 함수들
- extract_final_answer: LLM의 응답을 parse하여 최종 결과만 추출 (정답과 비교하기 위해)
- run_benchmark_test: 벤치마크 테스트
- save_final_result: 결과물 제출을 위한 함수

In [6]:
### 수정해도 됩니다!
def extract_final_answer(response: str):
    regex = r"(?:Answer:|Model response:)\s*\$?([0-9,]+)\b|([0-9,]+)\s*(meters|cups|miles|minutes)"
    matches = re.finditer(regex, response, re.MULTILINE)
    results = [match.group(1) if match.group(1) else match.group(2).replace(",", "") for match in matches]

    if len(results) == 0:
        additional_regex = r"\$?([0-9,]+)"
        additional_matches = re.finditer(additional_regex, response, re.MULTILINE)
        results.extend([match.group(1).replace(",", "") for match in additional_matches])

    return results[-1] if results else None

In [9]:
### 수정해도 됩니다!
def run_benchmark_test(
        dataset,
        prompt: str,
        model: str = "llama3-8b-8192",
        num_samples: int = 50,
        VERBOSE: bool = False
    ):
    correct = 0
    total   = 0
    results = []

    for i in tqdm(range(min(num_samples, len(dataset)))):
        question = dataset[i]["question"]
        correct_answer = float(re.findall(r'\d+(?:\.\d+)?', dataset[i]["answer"].split('####')[-1])[0])

        response = generate_response_using_Llama(
            prompt=prompt.format(question=question),
            model=model
        )

        if response:
            if VERBOSE:
                print("="*50)
                print(response)
                print("="*50)
            predicted_answer = extract_final_answer(response)

            if isinstance(predicted_answer, str):
                predicted_answer = float(predicted_answer.replace(",", ""))
            
            diff = abs(predicted_answer - correct_answer)
            is_correct = diff < 1e-5 if predicted_answer is not None else False
            
            if is_correct:
                correct += 1
            total += 1
            
            results.append({
                'question': question,
                'correct_answer': correct_answer,
                'predicted_answer': predicted_answer,
                'response': response,
                'correct': is_correct
            })

            if (i + 1) % 5 == 0:
                current_acc = correct/total if total > 0 else 0
                print(f"Progress: [{i+1}/{num_samples}]")
                print(f"Current Acc.: [{current_acc:.2%}]")

    return results, correct/total if total > 0 else 0

In [10]:
def save_final_result(results: List[Dict[str, Any]], accuracy: float, filename: str) -> None:
    result_str = f"====== ACCURACY: {accuracy} ======\n\n"
    result_str += f"[Details]\n"
    
    for idx, result in enumerate(results):
        result_str += f"Question {idx+1}: {result['question']}\n"
        result_str += f"Correct Answer: {result['correct_answer']}\n"
        result_str += f"Predicted Answer: {result['predicted_answer']}\n"
        result_str += f"Correct: {result['correct']}\n\n"
    
    with open(filename, "w", encoding="utf-8") as f:
        f.write(result_str)

#### Direct prompting with few-shot example

In [ ]:
def construct_direct_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )

    prompt = "Instruction:\nSolve the following mathematical question and generate ONLY the answer after a tag, 'Answer:' without any rationale.\n"

    for i in range(num_examples):
        cur_question = train_dataset['question'][i]
        cur_answer = train_dataset['answer'][i].split("####")[-1].strip()

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [ ]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_direct_prompt(3)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=10
)
save_final_result(results, accuracy, "example.txt")

  0%|          | 0/10 [00:00<?, ?it/s]

 50%|█████     | 5/10 [00:02<00:02,  2.08it/s]

Progress: [5/10]
Current Acc.: [20.00%]


100%|██████████| 10/10 [00:05<00:00,  1.96it/s]

Progress: [10/10]
Current Acc.: [20.00%]


In [ ]:
# TODO: 0 shot, 3 shot, 5 shot direct prompting을 통해 벤치마크 테스트를 한 후, 각각 direct_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 direct_prompting_5.txt
# 항상 num_samples=50 입니다!

# 0 shot
prompt_0_shot = construct_direct_prompt(0)
results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=prompt_0_shot,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "direct_prompting_0.txt")

 10%|█         | 5/50 [00:03<00:25,  1.73it/s]

Progress: [5/50]
Current Acc.: [40.00%]


 20%|██        | 10/50 [00:05<00:19,  2.04it/s]

Progress: [10/50]
Current Acc.: [30.00%]


 30%|███       | 15/50 [00:08<00:18,  1.94it/s]

...

Progress: [40/50]
Current Acc.: [20.00%]


 90%|█████████ | 45/50 [00:56<00:12,  2.41s/it]

Progress: [45/50]
Current Acc.: [20.00%]


100%|██████████| 50/50 [01:07<00:00,  1.36s/it]

Progress: [50/50]
Current Acc.: [22.00%]


In [ ]:
# 3 shot
prompt_3_shot = construct_direct_prompt(3)
results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=prompt_3_shot,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "direct_prompting_3.txt")

 10%|█         | 5/50 [00:02<00:18,  2.40it/s]

Progress: [5/50]
Current Acc.: [20.00%]


 20%|██        | 10/50 [00:14<01:25,  2.15s/it]

Progress: [10/50]
Current Acc.: [30.00%]


 30%|███       | 15/50 [00:26<01:22,  2.36s/it]

...

Progress: [40/50]
Current Acc.: [15.00%]


 90%|█████████ | 45/50 [01:40<00:12,  2.44s/it]

Progress: [45/50]
Current Acc.: [15.56%]


100%|██████████| 50/50 [01:52<00:00,  2.25s/it]

Progress: [50/50]
Current Acc.: [18.00%]


In [ ]:
# 5 shot
prompt_5_shot = construct_direct_prompt(5)
results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=prompt_5_shot,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "direct_prompting_5.txt")

 10%|█         | 5/50 [00:12<01:50,  2.45s/it]

Progress: [5/50]
Current Acc.: [40.00%]


 20%|██        | 10/50 [00:25<01:40,  2.50s/it]

Progress: [10/50]
Current Acc.: [20.00%]


 30%|███       | 15/50 [00:37<01:26,  2.47s/it]

...

Progress: [40/50]
Current Acc.: [20.00%]


 90%|█████████ | 45/50 [02:12<00:18,  3.74s/it]

Progress: [45/50]
Current Acc.: [17.78%]


100%|██████████| 50/50 [02:32<00:00,  3.05s/it]

Progress: [50/50]
Current Acc.: [18.00%]


### Chain-of-Thought prompting with few-shot example
```text
[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
====================================================================================================
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18
```

[Answer] 아래의 정답을 도출하는 과정을 예시로 달아주면 CoT의 few shot이 되겠죠?

In [ ]:
def construct_CoT_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )
    prompt = "You are a helpful assistatnt.\n Solve the following mathematical question step by step, and generate ONLY the final answer after a tag, 'Answer:'. \n" 

    for i in range(num_examples):
        #TODO: CoT example을 만들어주세요!
        cur_question = train_dataset['question'][sampled_indices[i]]
        thinking_steps = train_dataset['answer'][sampled_indices[i]].split("####")[0].strip()
        cur_answer = train_dataset['answer'][sampled_indices[i]].split("####")[-1].strip()
        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:\n{thinking_steps}\nAnswer: {cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"
    
    return prompt

In [ ]:
# TODO: 0 shot, 3 shot, 5 shot CoT prompting을 통해 벤치마크 테스트를 한 후, 각각 CoT_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 CoT_prompting_5.txt
# 항상 num_samples=50 입니다!

# 0 shot
prompt_0_shot = construct_CoT_prompt(0)
results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=prompt_0_shot,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "CoT_prompting_0.txt")

 10%|█         | 5/50 [00:02<00:21,  2.05it/s]

Progress: [5/50]
Current Acc.: [80.00%]


 20%|██        | 10/50 [00:04<00:18,  2.19it/s]

Progress: [10/50]
Current Acc.: [70.00%]


 30%|███       | 15/50 [00:07<00:16,  2.16it/s]

...

Progress: [40/50]
Current Acc.: [75.00%]


 90%|█████████ | 45/50 [01:12<00:15,  3.02s/it]

Progress: [45/50]
Current Acc.: [73.33%]


100%|██████████| 50/50 [01:28<00:00,  1.76s/it]

Progress: [50/50]
Current Acc.: [74.00%]


### Construct your prompt!!

목표: 본인만의 프롬프트를 통해 정답률을 더 끌어올려보기!
- gsm8k의 train 데이터셋에서 예시를 가져온 다음 (자유롭게!)
- 그 예시들에 대한 풀이 과정을 만들어주세요!
- 모든 것들이 자유입니다! Direct Prompting, CoT Prompting을 한 결과보다 정답률만 높으면 돼요.

In [11]:
def generate_response_using_Llama(
        prompt: str,
        model: str = "llama3-8b-8192"
    ):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant that solves math problems."
                },
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            model=model,
            temperature=0.7, ### 수정해도 됩니다!
            stream=False
        )
        return chat_completion.choices[0].message.content
    
    except Exception as e:
        print(f"API call error: {str(e)}")
        return None

In [12]:
from typing import List, Dict, Any

def save_self_consistent_result(results: List[Dict[str, Any]], accuracy: float, filename: str) -> None:
    result_str = f"====== ACCURACY (Self-Consistent): {accuracy:.4f} ======\n\n"
    result_str += f"[Details]\n"
    
    for idx, result in enumerate(results):
        result_str += f"Question {idx+1}:\n"
        result_str += f"Q: {result['question']}\n"
        result_str += f"Correct Answer: {result['correct_answer']}\n"
        result_str += f"Sampled Predictions: {result['prediction']}\n"
        result_str += f"Final (Majority) Answer: {result['final_prediction']}\n"
        result_str += f"Correct: {result['correct']}\n\n"
    
    with open(filename, "w", encoding="utf-8") as f:
        f.write(result_str)


In [13]:
from collections import Counter

def most_frequent(lst):
    return Counter(lst).most_common(1)[0][0] if lst else None

def run_self_consistent_test(
    dataset,
    prompt: str,
    model: str = "llama3-8b-8192",
    num_samples: int = 50,
    k: int = 5  # 몇 번 반복할지
):
    correct = 0
    total = 0
    results = []

    for i in tqdm(range(min(num_samples, len(dataset)))):
        question = dataset[i]["question"]
        correct_answer = float(re.findall(r'\d+(?:\.\d+)?', dataset[i]["answer"].split('####')[-1])[0])

        predictions = []
        for _ in range(k):
            response = generate_response_using_Llama(prompt.format(question=question), model=model)
            if response is None:
                continue
            parsed = extract_final_answer(response)
            if parsed:
                try:
                    predictions.append(float(parsed.replace(",", "")))
                except:
                    continue

        majority = most_frequent(predictions)

        is_correct = abs(majority - correct_answer) < 1e-5 if majority is not None else False
        if is_correct:
            correct += 1
        total += 1

        results.append({
            'question': question,
            'correct_answer': correct_answer,
            'prediction': predictions,
            'final_prediction': majority,
            'correct': is_correct
        })

        if (i + 1) % 5 == 0:
            current_acc = correct / total if total > 0 else 0
            print(f"Progress: [{i+1}/{num_samples}]")
            print(f"Current Self-Consistent Acc.: [{current_acc:.2%}]")

    return results, correct / total if total > 0 else 0


In [14]:
### 자유롭게 수정해도 됩니다! 완전히 새로 함수를 만들어도 돼요.
def construct_my_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train
    sampled_indices = random.sample(range(len(train_dataset)), num_examples)

    prompt = (
        "You are a brilliant and careful math tutor. "
        "For each problem, explain your reasoning step-by-step, verify your answer, and you MUST CONCLUDE with 'Answer:'.\n"
    )

    for i, idx in enumerate(sampled_indices):
        question = train_dataset[idx]['question']
        reasoning = train_dataset[idx]['answer'].split("####")[0].strip()
        final_answer = train_dataset[idx]['answer'].split("####")[-1].strip()

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{question}\n"
        prompt += f"Answer:\n{reasoning}\nDouble-checked Answer: {final_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"
    return prompt


In [ ]:
# TODO: 만든 0 shot, 3 shot, 5 shot example과 프롬프트를 통해 벤치마크 테스트를 한 후, 각각 My_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 My_prompting_5.txt
# 항상 num_samples=50 입니다!

# 0 shot
prompt_0_shot = construct_my_prompt(0)
results, accuracy = run_self_consistent_test(
    dataset=gsm8k_test,
    prompt=prompt_0_shot,
    num_samples=50
)
save_self_consistent_result(results, accuracy, "My_prompting_0.txt")

In [18]:
# 3 shot
prompt_3_shot = construct_my_prompt(3)
results, accuracy = run_self_consistent_test(
    dataset=gsm8k_test,
    prompt=prompt_3_shot,
    num_samples=50
)
save_self_consistent_result(results, accuracy, "My_prompting_3.txt")

 10%|█         | 5/50 [03:29<31:56, 42.58s/it]

Progress: [5/50]
Current Self-Consistent Acc.: [80.00%]


 20%|██        | 10/50 [07:16<30:21, 45.55s/it]

Progress: [10/50]
Current Self-Consistent Acc.: [70.00%]


 30%|███       | 15/50 [11:05<26:52, 46.06s/it]

...

Progress: [40/50]
Current Self-Consistent Acc.: [80.00%]


 90%|█████████ | 45/50 [32:55<03:39, 43.84s/it]

Progress: [45/50]
Current Self-Consistent Acc.: [82.22%]


100%|██████████| 50/50 [36:35<00:00, 43.91s/it]

Progress: [50/50]
Current Self-Consistent Acc.: [84.00%]


In [15]:
# 5 shot
prompt_5_shot = construct_my_prompt(5)
results, accuracy = run_self_consistent_test(
    dataset=gsm8k_test,
    prompt=prompt_5_shot,
    num_samples=50
)
save_self_consistent_result(results, accuracy, "My_prompting_5.txt")

  0%|          | 0/50 [00:00<?, ?it/s]

 10%|█         | 5/50 [05:06<52:10, 69.58s/it]

Progress: [5/50]
Current Self-Consistent Acc.: [80.00%]


 20%|██        | 10/50 [11:27<50:11, 75.29s/it]

Progress: [10/50]
Current Self-Consistent Acc.: [70.00%]


...

Progress: [40/50]
Current Self-Consistent Acc.: [75.00%]


 90%|█████████ | 45/50 [55:19<06:15, 75.05s/it]

Progress: [45/50]
Current Self-Consistent Acc.: [77.78%]


100%|██████████| 50/50 [1:01:33<00:00, 73.86s/it]

Progress: [50/50]
Current Self-Consistent Acc.: [78.00%]


### 보고서 작성하기
#### 아래의 내용이 포함되면 됩니다!

1. Direct Prompting, CoT Prompting, My Prompting을 0 shot, 3 shot, 5 shot 정답률을 표로 보여주세요!
2. CoT Prompting이 Direct Prompting에 비해 왜 좋을 수 있는지에 대해서 서술해주세요!
3. 본인이 작성한 프롬프트 기법이 CoT에 비해서 왜 더 좋을 수 있는지에 대해서 설명해주세요!
4. 최종적으로, `PROMPTING.md`에 보고서를 작성해주세요!